# Cross-approximation

Often, we would like to build a $N$-dimensional tensor from a **black-box function** $f: \Omega \subset \mathbb{R}^N \to \mathbb{R}$, where $\Omega$ is a tensor product grid. That is, we are free to sample *whatever entries we want* within our domain $\Omega$, but we cannot afford to sample the entire domain (as it contains an **exponentially large number of points**). One way to build such a tensor is using *cross-approximation* ([I. Oseledets, E. Tyrtyshnikov: "TT-cross Approximation for Multidimensional Arrays"](http://www.mat.uniroma2.it/~tvmsscho/papers/Tyrtyshnikov5.pdf)) from well-chosen fibers in the domain.

We support two major use cases of cross-approximation in the TT format.

## Approximating a Function over a Domain

This is the more basic setting. We just need to specify:

- Our function of interest
- The tensor product domain $\Omega = u_1 \otimes \dots \otimes u_N$
- The number of TT ranks

In [27]:
import tntorch as tn
import torch

def function(Xs):  # The function should accept a matrix (one row per sample, one column per input variable) and return a vector with one result per sample
    return 1./torch.sum(Xs, dim=1)  # Hilbert tensor

domain = [torch.arange(1, 33) for n in range(10)]
t = tn.cross(function=function, domain=domain, ranks_tt=8)

print(t)

Cross-approximation over a 10D domain:
iter: 0 | eps: 3.559e-07 | total time:   0.0769
iter: 1 | eps: 1.565e-06 | total time:   0.1401
iter: 2 | eps: 2.162e-06 | total time:   0.2166 <- converged
Did 101376 function evaluations in 0.00391173s (2.59159e+07 evals/s)

10D TT tensor:

 32  32  32  32  32  32  32  32  32  32
  |   |   |   |   |   |   |   |   |   |
 (0) (1) (2) (3) (4) (5) (6) (7) (8) (9)
 / \ / \ / \ / \ / \ / \ / \ / \ / \ / \
1   8   8   8   8   8   8   8   8   8   1



## Element-wise Operations on Tensors

Here we have one (or several) $N$-dimensional tensors that we want to transform element-wise. For instance, we may want to square each element of our tensor:

In [24]:
t2 = tn.cross(function=lambda x: x**2, tensors=t, ranks_tt=8)

Cross-approximation over a 10D domain:
iter: 0 | eps: 1.321e-05 | total time:   0.0401
iter: 1 | eps: 4.380e-05 | total time:   0.0738
iter: 2 | eps: 5.497e-05 | total time:   0.1173 <- converged
Did 101376 function evaluations in 0.00066781s (1.51804e+08 evals/s)



Just for practice, let's do this now in a slightly different way by passing two tensors:

In [25]:
t2 = tn.cross(function=lambda x: x[:, 0]*x[:, 1], tensors=[t, t], ranks_tt=8)

Cross-approximation over a 10D domain:
iter: 0 | eps: 2.436e-05 | total time:   0.0372
iter: 1 | eps: 3.643e-05 | total time:   0.0764
iter: 2 | eps: 4.644e-05 | total time:   0.1144 <- converged
Did 101376 function evaluations in 0.00103092s (9.83353e+07 evals/s)



Let's check the accuracy of our cross-approximated squaring operation, compared to the groundtruth `t*t`:

In [26]:
tn.relative_error(t*t, t2)

tensor(0.0000)